## Windows System Programming through PowerShell
Beyond the providers and cmdlets that PowerShell already ships with, it is possible to extend PowerShells capabilities out and interact with deeper and lower level components in the operating system. This is a deep topic and what is contained in this notebook is a very shallow treatment of what is possible.

## Windows Management Instrumentation (WMI)
WMI is the Windows specific implementation of the Common Information Model or CIM. The CIM is a standard for describing the structure of networked systems and their behaviour and is the basis for repeatable remote management of those systems. For example, a CIM class exists which describes the processes on a system, and each Windows machine will maintain a WMI repository of objects of that class; via the WMI subsystem, these objects and the processes they represent, can be interacted with.

In the following example, the `Get-CimInstance` cmdlet retrieves an instance of the _Win32_logicaldisk_ class from the WMI repository. It uses the `-query` parameter and an SQL-like syntax called WQL (Wmi query language) as the parameter value, which more or less reads as "retrieve every disc you know about" - and this is passed to the instance of the win32 disk class. The results are passed to the `Select-Object` cmdlet with some custom expressions to make them more readable.
<div class="alert alert-block alert-info"> 
<b>Side note: Old vs New WMI</b></br> 
WMI repositories populated by WMI providers is actually the 'old' way of implementing WMI. Newer versions of WMI, now simply called MI, use a cient server model to query machines more or less live, rather than from information stored in a WMI repo.

In [33]:
Get-CimInstance -Query "SELECT * FROM Win32_logicaldisk" 
| Select-Object DeviceID, 
    @{name="Size";expression={[math]::Round($_.Size/1GB,2)}},
    @{name="Available";expression={[math]::Round($_.FreeSpace/1GB,2)}}


DeviceID    Size Available
--------    ---- ---------
C:       237.600   152.890



## Add-Type and Win32 Functions
The `Add-Type` cmdlet has already been shown briefly in the [Binary Cmdlets](./10.%20Binary%20Cmdlets.ipynb) notebook. `Add-Type` allows for on-the-fly compilation of C# code, usually for the construction of custom .NET types. _Platform Invoke_ is a built in .NET mechanism for calling unmanaged code from managed code. This makes it possible to call the native Windows API functions from .NET, and therefore from PowerShell.

<div class="alert alert-block alert-info"> 
<b>Side note: Unmanaged vs Managed Code</b><br>
Managed code is code that after compilation, is run from a special application known as a _managed runtime environment_. The runtime environment can then handle things like memory management and hardware io, making programs safer and more stable. Unmanaged code runs directly on the machine after being compiled into an executable and is sometimes called 'native' code for this reason.
</div>

In the PowerShell code below, the unmanaged code DLL that contains the function we are after is first loaded via the `DLLImport` attribute, then we provide a .NET definition that matches the definition of the function from DLL. This declares what is essentially a .NET 'wrapper' around the native API function `MessageBox`. The defintion is copied from [pinvoke.net](https://www.pinvoke.net/). This is a trivial example but the possibilites should be clear.

In [ ]:
$signature = @'
[DllImport("user32.dll", SetLastError = true, CharSet= CharSet.Auto)]
public static extern int MessageBox(
    IntPtr hWnd, 
    String text, 
    String caption, 
    uint type);
'@
 
 $myType = Add-Type -MemberDefinition $signature -Name MsgBox32 -Namespace SystemProgNotebook -PassThru
 $myType::MessageBox(0, "Flipmode is the greatest", "System Programming with PowerShell",0)

1


## COM Objects
While a thorough exploration of COM is outside the scope of this notebook, it is worth highlighting how easy it is for PowerShell to utilize COM. The Component Object Model (COM) is a standard that many Windows applications adhere to; it requires a program to expose a set of functions, also called an _interface_ through which other programs can interact with it. Programs in this context are referred to as _COM Objects_.

The COM bjects that are registered on a system are stored in the registry. Querying the registry reveals what programs are COM compliant and can be interacted with via PowerShell.

In [46]:
Get-ChildItem HKLM:\Software\Classes | Where-Object { $_.PSChildName -match '^Shell.[a-zA-Z]*$' } | Select-Object PSChildName


PSChildName
-----------
Shell.Application
Shell.Autoplay
Shell.AutoPlayDirect
Shell.CDBurn
Shell.Explorer
Shell.FolderView
Shell.HWEventHandlerShellExecute
Shell.UIHelper



For example the Windows desktop shell itself is COM compliant, and methods from its Shell.Application object can be invoked via PowerShell:

In [21]:
$shell = New-Object -COMObject "Shell.Application"
$shell.Open("C:\")